In [1]:
!wget https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
!wget https://www.dropbox.com/s/ripsc7hfc0nihy5/test.txt

--2018-10-18 16:23:49--  https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/laax0sxqzzlddtq/ferdosi.txt [following]
--2018-10-18 16:23:49--  https://www.dropbox.com/s/raw/laax0sxqzzlddtq/ferdosi.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6168967961c31a6bd0d789ea68.dl.dropboxusercontent.com/cd/0/inline/ATWFhq9Uh8dQdM2vioCERj35fo7jS2kkH4Eb8guh7ts8gjI136bHcSNplxOZmA95RxB-POjHQ-PC8h-GSywmWY08mKkHSwQ5lA-T17f__CrkadDE22SnDi2oJxeYVgLS2wOgjv78yHJElpfQvTTmmN2WuDeSUsaILYVsFpxvlkw9x3xnxS4AbkQ4WqLxUGeey8Q/file [following]
--2018-10-18 16:23:50--  https://uc6168967961c31a6bd0d789ea68.dl.dropboxusercontent.com/cd/0/inline/ATWFhq9Uh8dQdM2vioCERj35fo7jS2kkH4Eb8guh7ts8gjI136b

In [2]:
import random
 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LSTM, GRU, CuDNNGRU, CuDNNLSTM 
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

from keras.losses import categorical_crossentropy
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
path = './ferdosi.txt'
with open(path, encoding='utf-8') as f:
    texts = f.read()

beits = texts.split('\n')
# texts = texts.split('\n')
texts = texts.replace('\n', ' ــ ')
texts = texts.split(' ')
# texts = text_to_word_sequence(texts)

print('corpus length:', len(texts))

corpus length: 660976


In [4]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
sequences = [sequences[i][0] for i in range(len(sequences)-1)]

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

len_of_words = len(word_index)
word2idx = word_index
idx2word = dict([(value, key) for (key, value) in word_index.items()])

# paded_sequences = pad_sequences(sequences, maxlen=max_length)

Found 19257 unique tokens.


In [5]:
max_length = 7
embeding_dim = 32
step = 8

sentences = []
next_word = []
for i in range(0, len(sequences) - max_length, step):
    sentences.append(sequences[i: i + max_length])
    next_word.append(sequences[i + max_length])
print('nb sequences:', len(sentences))

nb sequences: 82621


In [6]:
for ii in range(3):

    print(sentences[ii])
    print([idx2word.get(i, '') for i in sentences[ii]])

    print(next_word[ii])
    print(idx2word[next_word[ii]])


[3, 84, 355, 116, 2, 102, 1]
['به', 'نام', 'خداوند', 'جان', 'و', 'خرد', 'ــ']
357
کزین
[684, 232, 3631, 1, 355, 84, 2]
['برتر', 'اندیشه', 'برنگذرد', 'ــ', 'خداوند', 'نام', 'و']
355
خداوند
[55, 1, 355, 668, 262, 427, 1]
['جای', 'ــ', 'خداوند', 'روزی', 'ده', 'رهنمای', 'ــ']
355
خداوند


In [0]:
x = np.array(sentences)
y = np.array(next_word)
# y = to_categorical(y)

In [0]:
def on_epoch_end(epoch, _):
    #if epoch % 10 != 0:
       # return
    
    start_index = random.randint(0, len(beits) - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        sentence = beits[start_index]
        print(sentence)
        generated = sentence
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(max_length * 10):
            x_pred = np.zeros((1, max_length), dtype=int)
            for t, word in enumerate(sentence.split(' ')):
                try:
        #         print(t, word)
                    x_pred[0, t] = word2idx[word]
                except:
                    pass

            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_word = idx2word[pred_id]

            generated = generated + ' ' + next_word
            sentence = ' '.join(sentence.split(' ')[1:])
            sentence = sentence + ' ' + next_word

        print(generated.replace('ــ', '\n'))

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
# model = models.Sequential()
# model.add(layers.Embedding(len(word_index), 100))
# model.add(layers.LSTM(128))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


model = Sequential()
model.add(Embedding(len_of_words, embeding_dim))
model.add(CuDNNGRU(256, return_sequences=True))
model.add(CuDNNGRU(256, return_sequences=True))
model.add(CuDNNGRU(256))
# model.add(GRU(256))
model.add(Dense(len_of_words, activation='softmax'))
 
model.compile(optimizer=RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
print(model.summary())
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
history = model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          616224    
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, None, 256)         222720    
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, None, 256)         394752    
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (None, 256)               394752    
_________________________________________________________________
dense_1 (Dense)              (None, 19257)             4949049   
Total params: 6,577,497
Trainable params: 6,577,497
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/60
58240/82621 [====================>.........] - ETA: 6s - loss: 7.1584 - acc: 0.1368

KeyboardInterrupt: ignored